In [23]:
import cv2
import numpy
import os

img_arr = []
red = [(0, 0, 200),(20, 20, 255)] # lower and upper 
blue = [(200,0,0),(255, 20, 20)]
dot_colors = [blue, red]

#change the directory as needed
directory = './Validation Test-20220415T052724Z/Validation Test/xray'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    
    img = cv2.imread(f)
    # apply medianBlur to smooth image before threshholding
    blur= cv2.medianBlur(img, 7) # smooth image by 7x7 pixels, may need to adjust a bit

    coord = [0, 0]

    for i in range(2):
        lower, upper = dot_colors[i]
        output = img.copy()
        # apply threshhold color to white (255,255, 255) and the rest to black(0,0,0)
        mask = cv2.inRange(output,lower,upper) 
        circles = cv2.HoughCircles(mask,cv2.HOUGH_GRADIENT,1,7,param1=10,param2=7,
                               minRadius=0,maxRadius=7)
        index = 0
        if circles is not None:
            # convert the (x, y) coordinates and radius of the circles to integers
            circles = numpy.round(circles[0, :]).astype("int")

            # loop over the (x, y) coordinates and radius of the circles
            for (x, y, r) in circles:
                # draw the circle in the output image, 
                #   then draw a rectangle corresponding to the center of the circle
                cv2.circle(output, (x, y), r, (255, 0, 255), 2)
                cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (255, 0, 255), -1)

                index = index + 1
                #print str(index) + " : " + str(r) + ", (x,y) = " + str(x) + ', ' + str(y)
            #print('No. of circles detected = {}'.format(index))
            coord[i] = index
    img_arr.append((tuple(coord), f))
            #cv2.imshow('Output', output)
            #cv2.waitKey(0)
img_arr.sort()
#print(img_arr)

In [24]:
import math

from PIL import Image

im_list = [Image.open(x[1]) for x in img_arr]

n_rows = max([x[0][0] for x in img_arr])
n_cols = max([x[0][1] for x in img_arr])

def combine_img(im_list, n_rows, n_cols, resample=Image.BICUBIC):

    max_height = max(im.height for im in im_list)
    max_width = max(im.width for im in im_list)

    total_width = (max_width) * n_cols  # compute total width
    total_height = (max_height) * n_rows

    dst = Image.new("RGB", (total_width, total_height))  # calculate the total height

    pos_x = 0
    pos_y = 0
    for i in range(len(im_list)):
        im = im_list[i]
        dst.paste(im, (pos_y, pos_x))

        pos_y += im.width

        if (i + 1) % n_cols == 0:
            # new row
            pos_x += im.height
            pos_y = 0

    #print(" total: ", total_width, total_height)
    return dst


combine_img(im_list, n_rows, n_cols).save(
    "output_image.jpg"
)